In [1]:
import pyLDAvis
import pandas as pd
import json
import numpy as np

import warnings
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', 150)
pd.set_option('display.max_rows', 250)
# np.set_printoptions(suppress=True)

In [2]:
# phi_topic_term_dists_file = "/opt/0.imaginea/rpx/spark-lda-vis/phi/part-00000"
# theta_doc_topics_dists_file = "/opt/0.imaginea/rpx/spark-lda-vis/theta/part-00000"
# # doc_length_file = "/tmp/lda-vis/doc-length/part-00000"
# vocab_file = "/opt/0.imaginea/rpx/spark-lda-vis/vocab/part-00000"
# term_freq_file = "/opt/0.imaginea/rpx/spark-lda-vis/ter-freq/part-00000"

In [3]:
phi_topic_term_dists_file = "/opt/0.imaginea/rpx/model/scalaLDAvis/phi/part-00000"
theta_doc_topics_dists_file = "/opt/0.imaginea/rpx/model/scalaLDAvis/theta/part-00000"
# doc_length_file = "/tmp/lda-vis/doc-length/part-00000"
vocab_file = "/opt/0.imaginea/rpx/model/scalaLDAvis/vocab/part-00000"
term_freq_file = "/opt/0.imaginea/rpx/model/scalaLDAvis/termFreq/part-00000"

### Reference http://nbviewer.ipython.org/github/bmabey/pyLDAvis/blob/master/notebooks/pyLDAvis_overview.ipynb

In [4]:
import glob
path = '/opt/0.imaginea/rpx/model/scalaLDAvis/theta'
thetaFiles = glob.glob(path + "/part*")
theta_with_size = pd.DataFrame()
list_ = []
for file_ in thetaFiles:
    df = pd.read_csv(file_, index_col=None, header=None)
    list_.append(df)
theta_with_size = pd.concat(list_)
theta_with_size = theta_with_size[theta_with_size[0] > 0]
phi = pd.read_csv(phi_topic_term_dists_file, header=None)
# theta_with_size = pd.read_csv(theta_doc_topics_dists_file, header=None)
theta = theta_with_size.ix[:,1:]
doc_length = theta_with_size.ix[:,0]
vocab = pd.read_csv(vocab_file, header=None)
term_freq = pd.read_csv(term_freq_file, header=None)

In [5]:
flatten = lambda l: [item for sublist in l for item in sublist]

In [6]:
data = {'topic_term_dists': phi.values.tolist(), 
            'doc_topic_dists': theta.values.tolist(),
            'doc_lengths': doc_length.astype("int64").values.tolist(),
            'vocab': flatten(vocab.values.tolist()),
            'term_frequency': flatten(term_freq.astype("int64").values.tolist())}


print('Topic-Term shape: %s' % str(np.array(data['topic_term_dists']).shape))
print('Doc-Topic shape: %s' % str(np.array(data['doc_topic_dists']).shape))

Topic-Term shape: (20, 50000)
Doc-Topic shape: (11314, 20)


In [7]:
lda_vis_data = pyLDAvis.prepare(**data)

In [8]:
pyLDAvis.enable_notebook()

In [61]:
# pyLDAvis.prepare(mds='mmds', **data)

In [10]:
pyLDAvis.save_json(lda_vis_data, "/tmp/ldavis.json")

In [11]:
pyLDAvis.save_html(lda_vis_data, "/tmp/ldavis.html")

In [12]:
pyLDAvis.urls

<module 'pyLDAvis.urls' from '/home/mageswarand/anaconda3/envs/tensorflow1.0/lib/python3.5/site-packages/pyLDAvis/urls.py'>

In [13]:
# !cat /home/mageswarand/anaconda3/envs/tensorflow1.0/lib/python3.5/site-packages/pyLDAvis/urls.py

# Reference exploration for Scala Porting
**Only for developers**

# LDA - Latent Dirichlet allocation
- https://en.wikipedia.org/wiki/Latent_Dirichlet_allocation

K(integer) = number of topics (e.g. 50)  
V(integer) = number of words in the vocabulary (e.g. 50,000 or 1,000,000)  
M(integer) = number of documents  
$N_{d=1\dots M} $(integer) = number of words in document d  
N(integer) = total number of words in all documents; sum of all $N_{d}$ values, i.e. $N=\sum _{d=1}^{M}N_{d}$  
Z = N-dimension vector of integers between 1 and K = identity of topic of all words in all documents  
W = N-dimension vector of integers between 1 and V = identity of all words in all documents  

# Dirichlet distribution
- https://en.wikipedia.org/wiki/Dirichlet_distribution

In [14]:
from pyLDAvis import *
from pyLDAvis._prepare import *


In [15]:
topic_term_dists = pyLDAvis._prepare._df_with_names(data['topic_term_dists'], 'topic', 'term')
#[K x V]
doc_topic_dists  = pyLDAvis._prepare._df_with_names(data['doc_topic_dists'], 'doc', 'topic')
#[M x K]
term_frequency   = pyLDAvis._prepare._series_with_name(data['term_frequency'], 'term_frequency')
#[V]
doc_lengths      = pyLDAvis._prepare._series_with_name(data['doc_lengths'], 'doc_length')
#[M]
vocab            = pyLDAvis._prepare._series_with_name(data['vocab'], 'vocab')
#[V]

In [16]:
topic_term_dists.shape#[K x V]

(20, 50000)

In [17]:
doc_topic_dists.shape#[M x K]

(11314, 20)

In [18]:
term_frequency.shape#[V]

(50000,)

In [19]:
doc_lengths.shape#[M]

(11314,)

In [20]:
vocab.shape#[V]

(50000,)

In [21]:
topic_freq  = (doc_topic_dists.T * doc_lengths).T.sum(axis=0) # elementwise multiplication and sum all the rows
print(topic_freq.shape)    #[K x M] * [M] = [K x M] = [M x K] = [M,]
# doc_topic_dists.T
# topic_freq
# (doc_topic_dists.T * doc_lengths).T

(20,)


In [22]:
topic_proportion = (topic_freq / topic_freq.sum()).sort_values(ascending=False)
print(topic_proportion.shape)

(20,)


In [23]:
topic_order      = topic_proportion.index
# reorder all data based on new ordering of topics
topic_freq       = topic_freq[topic_order]
topic_term_dists = topic_term_dists.ix[topic_order]
doc_topic_dists  = doc_topic_dists[topic_order]

In [24]:
topic_order

Int64Index([3, 15, 8, 4, 12, 5, 19, 10, 7, 11, 13, 2, 0, 18, 16, 1, 17, 14, 6,
            9],
           dtype='int64', name='topic')

In [25]:
# token counts for each term-topic combination (widths of red bars)
term_topic_freq = (topic_term_dists.T * topic_freq).T

In [26]:
term_frequency = np.sum(term_topic_freq, axis=0)

# Relevance Calculation


In [69]:
def _find_relevance(log_ttd, log_lift, R, lambda_):
    relevance = lambda_ * log_ttd + (1 - lambda_) * log_lift #TODO log added here
    return relevance.T.apply(lambda s: s.sort_values(ascending=False).index).head(R)


def _find_relevance_chunks(log_ttd, log_lift, R, lambda_seq):
    return pd.concat([_find_relevance(log_ttd, log_lift, R, l) for l in lambda_seq])


Let $\phi_{wk}$ (topic_term_dists) denote the probability of term $w \in {1, ..., V }$ for topic $k \in {1, ..., K}$

Let $p_w$ (term_proportion) denote the marginal probability of term $w$ in the corpus.

Relevance of term $w$ to topic $k$ given a weight parameter $\lambda$ (where $0 \leq \lambda \leq 1$) as:

$$
r(w,k\ |\ \lambda) = \lambda \log(\phi_{wk}) + \log(1-\lambda)\log(\frac{\phi_{wk}}{p_w})
$$

## Saliency Calculation

distinctiveness ($w$) = $\sum_T\ P(T\vert w)\ \log(\frac{P(T\vert w)}{P(T)})$ 

saliency($w$) = $P(w) * distinctiveness ($w$)$

$P(T\vert w)$  ---> topic_given_term  
$P(T)$  ---> topic_proportion   
$P(w)$ ---> term_proportion    

In [81]:
print(topic_term_dists.shape)
# topic_term_dists

(20, 50000)


In [84]:
print(topic_proportion.shape)
# topic_proportion

(20,)


In [86]:
print(term_proportion.shape)
# term_proportion

(50000,)


In [70]:
R=30
lambda_step = 0.01
n_jobs = -1

# marginal distribution over terms (width of blue bars)
term_proportion = term_frequency / term_frequency.sum()

# compute the distinctiveness and saliency of the terms:
# this determines the R terms that are displayed when no topic is selected
topic_given_term = topic_term_dists / topic_term_dists.sum()
kernel = (topic_given_term * np.log((topic_given_term.T / topic_proportion).T))
distinctiveness = kernel.sum()
saliency = term_proportion * distinctiveness

# Order the terms for the "default" view by decreasing saliency:
default_term_info  = pd.DataFrame({'saliency': saliency, 'Term': vocab, \
                                  'Freq': term_frequency, 'Total': term_frequency, \
                                  'Category': 'Default'}). \
                                  sort_values(by='saliency', ascending=False)\
                                    .head(R).drop('saliency', 1)
# Rounding Freq and Total to integer values to match LDAvis code:
default_term_info['Freq'] = np.floor(default_term_info['Freq'])
default_term_info['Total'] = np.floor(default_term_info['Total'])
ranks = np.arange(R, 0, -1)
default_term_info['logprob'] = default_term_info['loglift'] = ranks

## compute relevance and top terms for each topic
log_lift = np.log(topic_term_dists / term_proportion)
log_ttd = np.log(topic_term_dists)

lambda_seq = np.arange(0, 1 + lambda_step, lambda_step)

def topic_top_term_df(tup):
    new_topic_id, (original_topic_id, topic_terms) = tup
    term_ix = topic_terms.unique()
#     print('===========')
#     print('new_topic_id: ', new_topic_id)
#     print('--------')
#     print('original_topic_id:' , original_topic_id)
#     print('--------')
#     print('term_ix: ', term_ix)
#     print('-========')

    return pd.DataFrame({'Term': vocab[term_ix], \
                   'Freq': term_topic_freq.loc[original_topic_id, term_ix], \
                   'Total': term_frequency[term_ix], \
                   'logprob': log_ttd.loc[original_topic_id, term_ix].round(4), \
                   'loglift': log_lift.loc[original_topic_id, term_ix].round(4), \
                   'Category': 'Topic%d' % new_topic_id})

top_terms = _find_relevance_chunks(log_ttd, log_lift, R, lambda_seq)

topic_dfs = map(topic_top_term_df, enumerate(top_terms.T.iterrows(), 1))
topic_info =  pd.concat([default_term_info] + list(topic_dfs))
topic_info

,Category,Freq,Term,Total,loglift,logprob
term,,,,,,
0,Default,1.891903e+09,would,1.891903e+09,30.0000,30.0000
561,Default,1.058966e+08,entry,1.058966e+08,29.0000,29.0000
9,Default,7.369618e+08,good,7.369618e+08,28.0000,28.0000
222,Default,8.684423e+07,israel,8.684423e+07,27.0000,27.0000
1,Default,1.441786e+09,article,1.441786e+09,26.0000,26.0000
218,Default,9.551117e+07,armenian,9.551117e+07,25.0000,25.0000
64,Default,2.782612e+08,data,2.782612e+08,24.0000,24.0000
573,Default,1.108963e+08,tape,1.108963e+08,23.0000,23.0000
2,Default,1.200430e+09,like,1.200430e+09,22.0000,22.0000


In [71]:
# def _token_table(topic_info, term_topic_freq, vocab, term_frequency):
# last, to compute the areas of the circles when a term is highlighted
# we must gather all unique terms that could show up (for every combination
# of topic and value of lambda) and compute its distribution over topics.

# term-topic frequency table of unique terms across all topics and all values of lambda
term_ix = topic_info.index.unique()
term_ix = np.sort(term_ix)

top_topic_terms_freq = term_topic_freq[term_ix]
# use the new ordering for the topics
K = len(term_topic_freq)
top_topic_terms_freq.index = range(1, K + 1)
top_topic_terms_freq.index.name = 'Topic'

# we filter to Freq >= 0.5 to avoid sending too much data to the browser
token_table = pd.DataFrame({'Freq': top_topic_terms_freq.unstack()}). \
             reset_index().set_index('term'). \
             query('Freq >= 0.5')

token_table['Freq'] = token_table['Freq'].round()
token_table['Term'] = vocab[token_table.index.values].values
# Normalize token frequencies:
token_table['Freq'] = token_table.Freq / term_frequency[token_table.index]
token_table = token_table.sort_values(by=['Term', 'Topic'])

print()
token_table


,Topic,Freq,Term
term,,,
43128,1,0.967625,aams
43128,2,0.004111,aams
43128,3,0.003496,aams
43128,4,0.003796,aams
43128,5,0.003314,aams
43128,6,0.002867,aams
43128,7,0.001883,aams
43128,8,0.001449,aams
43128,9,0.001624,aams


In [72]:
# def _topic_coordinates(mds, topic_term_dists, topic_proportion):
mds = js_PCoA
K = topic_term_dists.shape[0]
mds_res = mds(topic_term_dists)
assert mds_res.shape == (K, 2)
mds_df = pd.DataFrame({'x': mds_res[:,0], 'y': mds_res[:,1], 'topics': range(1, K + 1), \
                      'cluster': 1, 'Freq': topic_proportion * 100})
# note: cluster (should?) be deprecated soon. See: https://github.com/cpsievert/LDAvis/issues/26
topic_coordinates = mds_df
topic_coordinates

,Freq,cluster,topics,x,y
topic,,,,,
3,97.008800,1,1,-0.348517,0.000007
15,0.435436,1,2,-0.025122,-0.010354
8,0.346500,1,3,0.004572,-0.000099
4,0.318263,1,4,-0.021231,-0.001939
12,0.306475,1,5,0.006340,0.005107
5,0.289028,1,6,-0.037141,0.007684
19,0.158362,1,7,0.025131,0.000222
10,0.142260,1,8,0.035692,0.000574
7,0.134786,1,9,0.012998,0.000063


In [73]:
client_topic_order = [x + 1 for x in topic_order]

In [74]:
plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}

In [75]:
class PreparedData(namedtuple('PreparedData', ['topic_coordinates', 'topic_info', 'token_table',\
                                               'R', 'lambda_step', 'plot_opts', 'topic_order'])):
    def to_dict(self):
       return {'mdsDat': self.topic_coordinates.to_dict(orient='list'),
               'tinfo': self.topic_info.to_dict(orient='list'),
               'token.table': self.token_table.to_dict(orient='list'),
               'R': self.R,
               'lambda.step': self.lambda_step,
               'plot.opts': self.plot_opts,
               'topic.order': self.topic_order}

    def to_json(self):
       return json.dumps(self.to_dict(), cls=NumPyEncoder)


In [88]:
# topic_coordinates, topic_info, token_table, R, lambda_step, plot_opts, client_topic_order

In [76]:
pp = PreparedData(topic_coordinates, topic_info, token_table, R, lambda_step, plot_opts, client_topic_order)
pp.to_dict()

{'R': 30,
 'lambda.step': 0.01,
 'mdsDat': {'Freq': [97.00879994204807,
   0.4354361478276577,
   0.3464995354726806,
   0.3182633232408052,
   0.3064748966975714,
   0.2890281431667019,
   0.15836154372005032,
   0.1422600099455641,
   0.13478614703456546,
   0.13313628147034892,
   0.08921914179556416,
   0.08785560699868901,
   0.08426275087201361,
   0.07394045879003487,
   0.07131318485225087,
   0.06983233223065118,
   0.06624641780365431,
   0.06586143934259048,
   0.06473866403290265,
   0.05368403265763613],
  'cluster': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
  'topics': [1,
   2,
   3,
   4,
   5,
   6,
   7,
   8,
   9,
   10,
   11,
   12,
   13,
   14,
   15,
   16,
   17,
   18,
   19,
   20],
  'x': [-0.3485166792384901,
   -0.025122416400392245,
   0.004571683144814341,
   -0.021230990532115326,
   0.006340137082920331,
   -0.03714062687110602,
   0.02513141162748021,
   0.03569158511894552,
   0.012997663646152732,
   0.008786757144501618,
   0.0

In [89]:
fileobj = open("/tmp/lda1.json", 'w')
json.dump(pp.to_dict(), fileobj, cls=NumPyEncoder)

In [36]:
# topic_coordinates.to_dict(orient='list')
# topic_info.to_dict(orient='list')
# token_table.to_dict(orient='list')
topic_order

Int64Index([3, 15, 8, 4, 12, 5, 19, 10, 7, 11, 13, 2, 0, 18, 16, 1, 17, 14, 6,
            9],
           dtype='int64', name='topic')

## PCA Implentation Exploration

In [37]:
def _jensen_shannon(_P, _Q):
    _M = 0.5 * (_P + _Q)
    e1 = entropy(_P, _M)
    e2 = entropy(_Q, _M)
    res = 0.5 * ( e1 + e2 )
#     print('e1 ', e1)
#     print('e2 ', e2)
    print('res ', res)
    return res



In [38]:
# topic_term_dists

In [39]:
ss = pdist(topic_term_dists, metric=_jensen_shannon)
ss.shape

res  0.315758437099
res  0.350692421525
res  0.320504734549
res  0.352710101597
res  0.300445976921
res  0.373062058602
res  0.384068719854
res  0.360003127279
res  0.355369999575
res  0.381917808886
res  0.380626004001
res  0.38007011176
res  0.382511074825
res  0.379285834826
res  0.382263591522
res  0.381949452039
res  0.395204945016
res  0.373419465259
res  0.384089480571
res  0.0182498817333
res  0.015821032099
res  0.0203119768556
res  0.0194113807798
res  0.0164932151245
res  0.0177437538643
res  0.0159195492113
res  0.0157639463177
res  0.0152587200264
res  0.0155026483272
res  0.0157052663852
res  0.0167737849896
res  0.0161542658806
res  0.0151810893313
res  0.0164982678929
res  0.0175931559632
res  0.015131867528
res  0.0161584645964
res  0.0171993139735
res  0.0153555612706
res  0.0197248606688
res  0.0115060928408
res  0.0116719703532
res  0.0122338487955
res  0.0124968869212
res  0.0108187042681
res  0.0109044296454
res  0.0105512457761
res  0.0108644626055
res  0.0106456

(190,)

In [40]:
print(ss)

[ 0.31575844  0.35069242  0.32050473  0.3527101   0.30044598  0.37306206
  0.38406872  0.36000313  0.35537     0.38191781  0.380626    0.38007011
  0.38251107  0.37928583  0.38226359  0.38194945  0.39520495  0.37341947
  0.38408948  0.01824988  0.01582103  0.02031198  0.01941138  0.01649322
  0.01774375  0.01591955  0.01576395  0.01525872  0.01550265  0.01570527
  0.01677378  0.01615427  0.01518109  0.01649827  0.01759316  0.01513187
  0.01615846  0.01719931  0.01535556  0.01972486  0.01150609  0.01167197
  0.01223385  0.01249689  0.0108187   0.01090443  0.01055125  0.01086446
  0.01064568  0.01085451  0.01101384  0.01126582  0.01066349  0.01058517
  0.01758228  0.01666606  0.01293025  0.0156089   0.01467564  0.0151048
  0.01409395  0.0140851   0.01422238  0.01501071  0.01438913  0.01442361
  0.01470343  0.01582982  0.0139689   0.01369284  0.01967493  0.01216828
  0.01237911  0.01301587  0.01387807  0.01205717  0.01203306  0.01180228
  0.01179099  0.0117943   0.01219943  0.01180065  0.

In [41]:
ss.shape[0]

d = int(np.ceil(np.sqrt(ss.shape[0] * 2)))

d

20

In [42]:
d * (d - 1) / 2

190.0

In [43]:
ss[:10]

array([ 0.31575844,  0.35069242,  0.32050473,  0.3527101 ,  0.30044598,
        0.37306206,  0.38406872,  0.36000313,  0.35537   ,  0.38191781])

In [44]:
pair_dists = pyLDAvis._prepare.squareform(ss)

In [45]:
pair_dists[:10]

array([[ 0.        ,  0.31575844,  0.35069242,  0.32050473,  0.3527101 ,
         0.30044598,  0.37306206,  0.38406872,  0.36000313,  0.35537   ,
         0.38191781,  0.380626  ,  0.38007011,  0.38251107,  0.37928583,
         0.38226359,  0.38194945,  0.39520495,  0.37341947,  0.38408948],
       [ 0.31575844,  0.        ,  0.01824988,  0.01582103,  0.02031198,
         0.01941138,  0.01649322,  0.01774375,  0.01591955,  0.01576395,
         0.01525872,  0.01550265,  0.01570527,  0.01677378,  0.01615427,
         0.01518109,  0.01649827,  0.01759316,  0.01513187,  0.01615846],
       [ 0.35069242,  0.01824988,  0.        ,  0.01719931,  0.01535556,
         0.01972486,  0.01150609,  0.01167197,  0.01223385,  0.01249689,
         0.0108187 ,  0.01090443,  0.01055125,  0.01086446,  0.01064568,
         0.01085451,  0.01101384,  0.01126582,  0.01066349,  0.01058517],
       [ 0.32050473,  0.01582103,  0.01719931,  0.        ,  0.01758228,
         0.01666606,  0.01293025,  0.0156089 ,  

In [46]:
pyLDAvis._prepare.js_PCoA(topic_term_dists)

array([[ -3.48516679e-01,   6.85799425e-06],
       [ -2.51224164e-02,  -1.03538547e-02],
       [  4.57168314e-03,  -9.94736053e-05],
       [ -2.12309905e-02,  -1.93871604e-03],
       [  6.34013708e-03,   5.10673027e-03],
       [ -3.71406269e-02,   7.68389479e-03],
       [  2.51314116e-02,   2.22057881e-04],
       [  3.56915851e-02,   5.73536674e-04],
       [  1.29976636e-02,   6.28844586e-05],
       [  8.78675714e-03,   1.14808066e-03],
       [  3.36041146e-02,  -1.08707131e-03],
       [  3.23579282e-02,  -7.81657824e-04],
       [  3.18266854e-02,  -2.90696359e-04],
       [  3.41817729e-02,   4.75996044e-04],
       [  3.10717228e-02,   2.60713229e-05],
       [  3.39364735e-02,  -1.50461551e-03],
       [  3.36402771e-02,   2.92083184e-04],
       [  4.66804519e-02,   4.49962276e-05],
       [  2.54773663e-02,   3.90318913e-04],
       [  3.57146826e-02,   2.25769494e-05]])

In [47]:
pair_dists = np.asarray(pair_dists, np.float64)

In [48]:
n = pair_dists.shape[0]
n

20

In [49]:
H = np.eye(n) - np.ones((n, n)) / n
H

array([[ 0.95, -0.05, -0.05, -0.05, -0.05, -0.05, -0.05, -0.05, -0.05,
        -0.05, -0.05, -0.05, -0.05, -0.05, -0.05, -0.05, -0.05, -0.05,
        -0.05, -0.05],
       [-0.05,  0.95, -0.05, -0.05, -0.05, -0.05, -0.05, -0.05, -0.05,
        -0.05, -0.05, -0.05, -0.05, -0.05, -0.05, -0.05, -0.05, -0.05,
        -0.05, -0.05],
       [-0.05, -0.05,  0.95, -0.05, -0.05, -0.05, -0.05, -0.05, -0.05,
        -0.05, -0.05, -0.05, -0.05, -0.05, -0.05, -0.05, -0.05, -0.05,
        -0.05, -0.05],
       [-0.05, -0.05, -0.05,  0.95, -0.05, -0.05, -0.05, -0.05, -0.05,
        -0.05, -0.05, -0.05, -0.05, -0.05, -0.05, -0.05, -0.05, -0.05,
        -0.05, -0.05],
       [-0.05, -0.05, -0.05, -0.05,  0.95, -0.05, -0.05, -0.05, -0.05,
        -0.05, -0.05, -0.05, -0.05, -0.05, -0.05, -0.05, -0.05, -0.05,
        -0.05, -0.05],
       [-0.05, -0.05, -0.05, -0.05, -0.05,  0.95, -0.05, -0.05, -0.05,
        -0.05, -0.05, -0.05, -0.05, -0.05, -0.05, -0.05, -0.05, -0.05,
        -0.05, -0.05],
       [-0

In [50]:
B = - H.dot(pair_dists ** 2).dot(H) / 2
B

array([[  1.20746690e-01,   9.93357834e-03,  -1.17595094e-03,
          8.47539259e-03,  -1.83963025e-03,   1.44327502e-02,
         -8.89965514e-03,  -1.28550961e-02,  -4.34489833e-03,
         -2.76866236e-03,  -1.20800075e-02,  -1.16117921e-02,
         -1.14120948e-02,  -1.22930691e-02,  -1.11274541e-02,
         -1.22037368e-02,  -1.20905909e-02,  -1.69808797e-02,
         -9.03280837e-03,  -1.28720843e-02],
       [  9.93357834e-03,  -1.17614238e-03,  -8.11308785e-04,
         -1.24953351e-03,  -8.05126578e-04,  -1.58317418e-03,
         -4.09434441e-04,  -2.19541687e-04,  -6.31904545e-04,
         -7.10411076e-04,  -2.27231443e-04,  -2.55296660e-04,
         -2.70193604e-04,  -2.37803878e-04,  -2.90477992e-04,
         -2.17658891e-04,  -2.45411361e-04,  -3.58099396e-06,
         -3.87662575e-04,  -2.01683751e-04],
       [ -1.17595094e-03,  -8.11308785e-04,  -1.13417008e-04,
         -7.40926502e-04,  -1.85372323e-04,  -1.05794571e-03,
          1.91746230e-04,   4.01123953e-04

In [51]:
eigvals, eigvecs = np.linalg.eig(B)
eigvecs

array([[ -9.38692395e-01,   2.62390657e-01,   4.82048997e-04,
          2.28165823e-03,  -1.03059595e-03,   3.79469898e-04,
          2.57129354e-04,  -9.01570932e-04,  -2.23606798e-01,
         -1.37435560e-04,   1.57585387e-04,  -2.61257438e-04,
          1.37317674e-04,  -6.52241988e-05,  -3.28181446e-05,
          1.68382413e-04,  -1.39974382e-04,   1.42498613e-04,
          3.57551254e-05,   3.66574816e-05],
       [ -6.76645413e-02,  -4.31009804e-01,  -7.27773325e-01,
         -3.14524299e-02,  -2.06410981e-01,  -3.64091811e-01,
         -3.54994254e-02,   1.14818388e-01,  -2.23606798e-01,
         -1.03449941e-01,   1.00572152e-01,  -3.83968256e-02,
         -9.19912828e-02,   4.98368308e-03,  -2.64142750e-02,
          6.08396940e-02,  -1.23334549e-02,   2.75082862e-03,
         -5.97356072e-02,  -2.21715025e-02],
       [  1.23133395e-02,  -1.52614137e-01,  -6.99200814e-03,
          3.90220500e-01,  -5.58483995e-01,   6.65423096e-01,
         -4.56215672e-02,   1.03918003e-01

In [52]:
ix = eigvals.argsort()[::-1][:2]
ix

array([0, 2])

In [53]:
eigvals = eigvals[ix]

In [54]:
eigvecs = eigvecs[:, ix]
eigvecs

array([[ -9.38692395e-01,   4.82048997e-04],
       [ -6.76645413e-02,  -7.27773325e-01],
       [  1.23133395e-02,  -6.99200814e-03],
       [ -5.71834020e-02,  -1.36272515e-01],
       [  1.70764810e-02,   3.58952503e-01],
       [ -1.00034306e-01,   5.40101616e-01],
       [  6.76887690e-02,   1.56084672e-02],
       [  9.61314666e-02,   4.03139414e-02],
       [  3.50078167e-02,   4.42015392e-03],
       [  2.36661905e-02,   8.06986865e-02],
       [  9.05090880e-02,  -7.64103341e-02],
       [  8.71526183e-02,  -5.49427946e-02],
       [  8.57217728e-02,  -2.04330717e-02],
       [  9.20649490e-02,   3.34578022e-02],
       [  8.36883616e-02,   1.83255550e-03],
       [  9.14042613e-02,  -1.05759552e-01],
       [  9.06064879e-02,   2.05305518e-02],
       [  1.25728804e-01,   3.16278865e-03],
       [  6.86205607e-02,   2.74355494e-02],
       [  9.61936773e-02,   1.58693569e-03]])

In [55]:
eigvals[np.isclose(eigvals, 0)] = 0
np.any(eigvals < 0)

False

In [56]:
if np.any(eigvals < 0):
    ix_neg = eigvals < 0
    eigvals[ix_neg] = np.zeros(eigvals[ix_neg].shape)
    eigvecs[:, ix_neg] = np.zeros(eigvecs[:, ix_neg].shape)

In [57]:
eigvals.shape
type(eigvals)

numpy.ndarray

In [58]:
eigvecs.shape

(20, 2)

In [59]:
np.sqrt(eigvals) * eigvecs


array([[ -3.48516679e-01,   6.85799425e-06],
       [ -2.51224164e-02,  -1.03538547e-02],
       [  4.57168314e-03,  -9.94736053e-05],
       [ -2.12309905e-02,  -1.93871604e-03],
       [  6.34013708e-03,   5.10673027e-03],
       [ -3.71406269e-02,   7.68389479e-03],
       [  2.51314116e-02,   2.22057881e-04],
       [  3.56915851e-02,   5.73536674e-04],
       [  1.29976636e-02,   6.28844586e-05],
       [  8.78675714e-03,   1.14808066e-03],
       [  3.36041146e-02,  -1.08707131e-03],
       [  3.23579282e-02,  -7.81657824e-04],
       [  3.18266854e-02,  -2.90696359e-04],
       [  3.41817729e-02,   4.75996044e-04],
       [  3.10717228e-02,   2.60713229e-05],
       [  3.39364735e-02,  -1.50461551e-03],
       [  3.36402771e-02,   2.92083184e-04],
       [  4.66804519e-02,   4.49962276e-05],
       [  2.54773663e-02,   3.90318913e-04],
       [  3.57146826e-02,   2.25769494e-05]])

In [60]:
eigvecs * np.sqrt(eigvals)

array([[ -3.48516679e-01,   6.85799425e-06],
       [ -2.51224164e-02,  -1.03538547e-02],
       [  4.57168314e-03,  -9.94736053e-05],
       [ -2.12309905e-02,  -1.93871604e-03],
       [  6.34013708e-03,   5.10673027e-03],
       [ -3.71406269e-02,   7.68389479e-03],
       [  2.51314116e-02,   2.22057881e-04],
       [  3.56915851e-02,   5.73536674e-04],
       [  1.29976636e-02,   6.28844586e-05],
       [  8.78675714e-03,   1.14808066e-03],
       [  3.36041146e-02,  -1.08707131e-03],
       [  3.23579282e-02,  -7.81657824e-04],
       [  3.18266854e-02,  -2.90696359e-04],
       [  3.41817729e-02,   4.75996044e-04],
       [  3.10717228e-02,   2.60713229e-05],
       [  3.39364735e-02,  -1.50461551e-03],
       [  3.36402771e-02,   2.92083184e-04],
       [  4.66804519e-02,   4.49962276e-05],
       [  2.54773663e-02,   3.90318913e-04],
       [  3.57146826e-02,   2.25769494e-05]])